In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#You must have data set and Glove pretrained Glove Embedding matrxi in given path

In [3]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.utils import resample
import matplotlib.pyplot as plt
import string

# Load your dataset
data = pd.read_csv("/content/gdrive/MyDrive/TweetDS/train.csv")
data['tweet'] = data['tweet'].astype(str)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['tweet'])
X = tokenizer.texts_to_sequences(data['tweet'])
X = pad_sequences(X, maxlen=100)
y = data['sarcastic']

# Oversample the minority class
data_majority = data[data.sarcastic == 0]
data_minority = data[data.sarcastic == 1]
data_minority_upsampled = resample(data_minority, replace=True, n_samples=len(data_majority), random_state=42)
data_upsampled = pd.concat([data_majority, data_minority_upsampled])

# Split the data into training and testing sets
X_upsampled = tokenizer.texts_to_sequences(data_upsampled['tweet'])
X_upsampled = pad_sequences(X_upsampled, maxlen=100)
y_upsampled = data_upsampled['sarcastic']
X_train, X_test, y_train, y_test = train_test_split(X_upsampled, y_upsampled, test_size=0.2, random_state=42)

# Load pre-trained GloVe word vectors
embedding_dim = 300
vocab_size = len(tokenizer.word_index) + 1
glove_path = "/content/gdrive/MyDrive/PreTrainedModels/GloVe/glove.6B.300d.txt"
embeddings_index = {}
with open(glove_path, encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
        embeddings_index[word] = coefs

# Create an embedding matrix for the Embedding layer
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Build the LSTM model with GloVe embeddings
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim,
                    input_length=100, weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model with Adam optimizer
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Implement early stopping and model checkpoint
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True, mode='max')
model_checkpoint = ModelCheckpoint("/content/gdrive/MyDrive/MyModels/sarcasm_detection_best_model.h5", save_best_only=True)

# Train the model and calculate metrics for each epoch
for iteration in range(5):
    history = model.fit(X_train, y_train, validation_split=0.2,
                        epochs=3, batch_size=64, callbacks=[early_stopping, model_checkpoint], verbose=2)

# Evaluate the model on the testing data
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
y_pred = (model.predict(X_test) > 0.5).astype(int)

# Calculate final metrics for the best model
final_precision = precision_score(y_test, y_pred)
final_recall = recall_score(y_test, y_pred)
final_f1 = f1_score(y_test, y_pred)

print(f'Final Precision = {final_precision:.4f}, Recall = {final_recall:.4f}, F1-score = {final_f1:.4f}')


Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


52/52 - 39s - loss: 0.6915 - accuracy: 0.5303 - val_loss: 0.6779 - val_accuracy: 0.6291 - 39s/epoch - 752ms/step
Epoch 2/3
52/52 - 28s - loss: 0.6549 - accuracy: 0.6187 - val_loss: 0.6203 - val_accuracy: 0.6651 - 28s/epoch - 535ms/step
Epoch 3/3
52/52 - 29s - loss: 0.5508 - accuracy: 0.7218 - val_loss: 0.5927 - val_accuracy: 0.7071 - 29s/epoch - 558ms/step
Epoch 1/3
52/52 - 34s - loss: 0.4217 - accuracy: 0.8134 - val_loss: 0.5375 - val_accuracy: 0.7431 - 34s/epoch - 651ms/step
Epoch 2/3
52/52 - 28s - loss: 0.2952 - accuracy: 0.8813 - val_loss: 0.5319 - val_accuracy: 0.7875 - 28s/epoch - 544ms/step
Epoch 3/3
52/52 - 29s - loss: 0.1763 - accuracy: 0.9324 - val_loss: 0.6042 - val_accuracy: 0.7959 - 29s/epoch - 552ms/step
Epoch 1/3
52/52 - 29s - loss: 0.1221 - accuracy: 0.9546 - val_loss: 0.6858 - val_accuracy: 0.8019 - 29s/epoch - 550ms/step
Epoch 2/3
52/52 - 29s - loss: 0.0748 - accuracy: 0.9718 - val_loss: 0.6480 - val_accuracy: 0.8403 - 29s/epoch - 552ms/step
Epoch 3/3
52/52 - 26s - lo

In [8]:
# Preprocess the input sentence
def preprocess_text(text):
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])
    return text

# Example input sentence
input_sentence = "Congratulations! Your ability to state the obvious is truly remarkable"

# Preprocess the input sentence
input_sentence = preprocess_text(input_sentence)

# Tokenize and pad the preprocessed sentence
input_sequence = tokenizer.texts_to_sequences([input_sentence])
input_sequence = pad_sequences(input_sequence, maxlen=100)

# Predict the sentiment of the input sentence
prediction = (model.predict(input_sequence) > 0.5).astype(int)[0][0]

# Interpret the prediction
if prediction == 1:
    print("The sentence is sarcastic.")
else:
    print("The sentence is not sarcastic.")

1/1 [==============================] - 0s 102ms/step
The sentence is sarcastic.
